## Домашнее задание

### 1. Загрузите тренировочные и тестовые датасеты

In [165]:
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from pathlib import Path
import io
from scipy import stats


In [26]:
dataset_train_filename = "TrainData.csv"
dataset_test_filename = "TrainData.csv"

df = pd.read_csv(dataset_train_filename)
df_delayed = pd.read_csv(dataset_test_filename)

In [27]:
# Сгруппируем основную информацию о датасете и выгрузим в excel.
# import io
dataset_filename = dataset_train_filename
info_filename = f'./{Path(dataset_filename).stem}_info.xlsx'
buf = io.StringIO()
df.info(buf=buf)
s = buf.getvalue()
columns = s.splitlines()[3].split()
lines =   [line.split() for line in s.splitlines()[5:-2]]
lines = [[' '.join(row[1:-3]), row[-3], row[-2], row[-1]   ] for row in lines]
df_info = pd.merge(
    pd.DataFrame(lines, columns=columns[1:])
    , df.nunique().to_frame(name='Nu'), 
    left_on='Column', 
    right_index=True)# .set_index(['#']).reset_index()
df_vc = pd.DataFrame([[col, sorted(df[col].value_counts().to_dict().items(), key=lambda x: x[0])] for col in df.columns if len(df[col].value_counts()) < 21], columns=['Column', 'vc'])
df_info = pd.merge(df_info, df_vc, how='left', on='Column')
df_info = pd.merge(df_info, df.describe().T, how='left', left_on='Column', right_index=True)
df_info = df_info[['Column', 'Non-Null']].copy()\
    .join(pd.DataFrame(df.isna().sum()).rename(columns={0: 'Null_cnt'}), how='left', on='Column')\
    .join(pd.DataFrame(df.isna().sum() / df.shape[0]).round(3).rename(columns={0: 'Null_percent'}), how='left', on='Column')\
    .join(df_info.set_index('Column').drop(columns=['Non-Null', 'Count'] ), how='left', on='Column')
if not Path(info_filename).exists():
    df_info.to_excel(info_filename, index=False)
df_info

,Column,Non-Null,Null_cnt,Null_percent,Dtype,Nu,vc,count,mean,std,min,25%,50%,75%,max
0,f1,7500,0,0.000,int64,70,NaN,7500.0,38.579600,13.647719,17.000000,28.000000,37.000000,47.000000,9.000000e+01
1,f2,7425,75,0.010,float64,8,"[(64.2590702793375, 2), (114.41647597254, 434)...",7425.0,238.905308,76.198282,64.259070,218.593930,218.593930,218.593930,5.753968e+02
2,f3,7500,0,0.000,int64,6602,NaN,7500.0,190384.986800,105867.442524,19302.000000,118019.000000,179568.500000,239441.750000,1.226583e+06
3,f4,7500,0,0.000,float64,16,"[(2.948307735566308e-05, 10), (0.0256410256410...",7500.0,0.238933,0.151738,0.000029,0.171817,0.179322,0.239316,7.293233e-01
4,f5,7500,0,0.000,int64,16,"[(1, 10), (2, 39), (3, 64), (4, 153), (5, 112)...",7500.0,10.083200,2.546071,1.000000,9.000000,10.000000,12.000000,1.600000e+01
5,f6,7500,0,0.000,float64,7,"[(0.0491270807957775, 2463), (0.06637168141592...",7500.0,0.238934,0.190283,0.049127,0.049127,0.098837,0.445026,4.450262e-01
6,f7,5625,1875,0.250,float64,15,"[(0.0, 34), (0.0449871465295629, 583), (0.0716...",5625.0,0.240556,0.147371,0.000000,0.115721,0.241573,0.439103,4.822222e-01
7,f8,7500,0,0.000,float64,6,"[(0.0094991364421416, 1158), (0.02304147465437...",7500.0,0.238933,0.193365,0.009499,0.066581,0.103024,0.444334,4.843750e-01
8,f9,7500,0,0.000,float64,5,"[(5.97014925373134, 67), (10.29411764705882, 6...",7500.0,23.893333,4.451530,5.970149,25.606721,25.606721,25.606721,2.560672e+01
9,f10,7500,0,0.000,float64,2,"[(0.1135902636916835, 2465), (0.30029791459781...",7500.0,0.238933,0.087708,0.113590,0.113590,0.300298,0.300298,3.002979e-01


In [28]:
# Работа с признаками
# 1. Признак f7 исключить т.к. пропусков 25% от объема датасета
# 2. Часть признаков можно отнести к категориальным т.к. они содержат ограниченный набор значений.
# Нет информации имеют ли категориальные признаки естественную порядковую связь (относятся ранговой или к номинальной шкале)
# т.к. датасет поступил уже предобарботанный, то предположим, что категориальные признаки относятся к ранговой шкале
# (т.к. возможности проверить и отвергнуть эту гипотезу нет).
# 3. Признак f11 можно заполнить 0 т.к. это самое часто встречающеся для признака значение.
# 4. Перед построением модели выполним масштабирование признаков т.к. для модели SVM, основанной на рассчете расстояний, 
# разный масштаб признаков может оказать влияние на результат

In [146]:
def rm_item_from_list(item:any, lst:list)->list:
    try:
        return lst.remove(item) or lst
    except ValueError:
        return lst

cat_features = ['f2', 'f4', 'f5', 'f6', 'f8', 'f9', 'f10']

class DataPreprocessing:
    def __init__(self):
        self.f2_by_other_cat_features = None
        self.f2_mode = None
        self.f11_mode = None
        return

    def fit(self, X:pd.DataFrame)->pd.DataFrame:
        # # f2_by_other_cat_features = df.groupby(['f5', 'f6', 'f8', 'f9', 'f10'])['f2'].transform(lambda x :x.fillna(stats.mode(x)[0][0]))
        # f2_by_other_cat_features = df.groupby(['f5', 'f6', 'f8', 'f9', 'f10']).agg({'f2':lambda x :stats.mode(x)[0][0]})
        # # f2_by_other_cat_features
        # df['f2'] = df['f2'].fillna(df.join(f2_by_other_cat_features['f2'], on=['f5', 'f6', 'f8', 'f9', 'f10'], how='left', rsuffix='_')['f2_']).value_counts()

        self.f11_mode = X['f11'].mode()[0]
        self.f2_mode = X['f2'].mode()[0]
        self.f2_by_other_cat_features = X.groupby(['f5', 'f6', 'f8', 'f9', 'f10']).agg({'f2':lambda x :stats.mode(x)[0][0]})


    def transform(self, source_df:pd.DataFrame)->pd.DataFrame:
        X = source_df.copy(deep=True)
        X.drop(columns='f7', inplace=True)
        X['f11'] = X['f11'].fillna(self.f11_mode)#.isna().value_counts()
        X['f2'] = X['f2'].fillna(X.join(self.f2_by_other_cat_features['f2'], on=['f5', 'f6', 'f8', 'f9', 'f10'], how='left', rsuffix='_')['f2_'])
        X['f2'] = X['f2'].fillna(self.f2_mode) 
        return X



In [137]:
data_preprocessing = DataPreprocessing()
data_preprocessing.fit(df)


C:\Users\Alex\AppData\Local\Temp\ipykernel_9976\2151295110.py:24: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  self.f2_by_other_cat_features = X.groupby(['f5', 'f6', 'f8', 'f9', 'f10']).agg({'f2':lambda x :stats.mode(x)[0][0]})


In [138]:
df = data_preprocessing.transform(df)
df.isna().sum()

f1        0
f2        0
f3        0
f4        0
f5        0
f6        0
f8        0
f9        0
f10       0
f11       0
f12       0
f13       0
f14       0
target    0
dtype: int64

### 2. Оцените баланс классов в задаче
- Затем попытайтесь устно ответить на вопрос, можно ли использовать accuracy как метрику качества в задаче?

In [139]:
df['target'].value_counts()

target
0    5708
1    1792
Name: count, dtype: int64

In [140]:
# Наблюдаем дисбаланс классов целевой переменной
# Вопрос про метрику accuracy не простой:
#  - Если более важным является правильное предсказание класса имеющего большую представленность, то метрика может быть использована.
#  - Если более важным является правильное предсказание класса имеющего меньшую представленность, то использовать метрику нежелательно.

### 3. Постройте baseline-модель:
- разбейте TrainData на тренировочные (Train) и тестовые данные (Test);
- обучите LogisticRegression и SVC с параметрами по умолчанию на тренировочных данных (Train);
- примените модели на тестовых данных (Test).

In [161]:
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y\
                                                    , train_size=0.8\
                                                    , random_state=1
                                                    ,stratify=df[['f9', 'f10']])
print(f'{[el.shape for el in [X_train, X_test, y_train, y_test]]}')

[(6000, 13), (1500, 13), (6000,), (1500,)]


In [162]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [163]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [168]:
model_base_svm = SVC()
model_base_svm.fit(X_train_scaled, y_train)

SVC()

In [169]:
y_test_base_svm = model_base_svm.predict(X_test)

c:\Users\Alex\Documents\GB\Python_2022\.python_b_hw\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [170]:
from sklearn.metrics import f1_score

In [171]:
f1_score(y_test, y_test_base_svm)
# Получили нулевую точность

0.0

### 4. Улучшите модели
Попробуйте улучшить качество обученных моделей:
- можете задавать class_weights;
- можете изменять параметры модели;
- можете вручную или при помощи методов Python генерировать новые признаки и/или удалять существующие.

Это самая важная и творческая часть задания. Проводите как можно больше экспериментов!

Проведите минимиум три эксперимента: для каждого типа модели минимум один эксперимент.

In [ ]:
# Ваш код здесь

### 5. Оцените на отложенной выборке качество наилучшей модели
В пунктах 3 и 4 вы построили много разных моделей.

Возьмите ту, которая дала наилучшее качество на тестовых данных (Test). Примените её на отложенной выборке (TestData) и выведите на экран значение метрики f1.

In [ ]:
# Ваш код здесь

### 6. Выполните хитрый трюк
Часто смешивание различных моделей даёт улучшение итогового предсказания. Попробуйте смешать две лучшие модели по формуле:
$$pred_{final} = \alpha\cdot pred_1 + (1-\alpha)\cdot pred_2$$.

Значение $\alpha$ подберите в цикле по Test-выборке. Оцените качество на отложенной выборке.

Удалось ли добиться улучшения качества?

In [ ]:
# Ваш код здесь

### 7. Сделайте выводы

Запишите в отдельной ячейке текстом выводы о проделанной работе. Для этого ответьте на вопросы:
- Какие подходы вы использовали для улучшения работы baseline-моделей?
- Какого максимального качества удалось добиться на Test-данных?
- Какое при этом получилось качество на отложенной выборке?
- Ваша модель переобучилась, недообучилась или обучилась как надо?

In [ ]:
# Ваш текст здесь

Важный комментарий! В реальных задачах не следует ожидать, что машинным обучением всегда удастся решить задачу с хорошим качеством. Но использовать все имеющиеся у вас в арсенале методы для достижения наилучшего результата нужно.